# Generate Endeavour gene lists (training and candidate)


Gerar antes o conversor Maf clean para já usar o MAF filtrado e limpo


In [ ]:
import pandas as pd
import os


In [ ]:
path_mafs_nCop = '/content/drive/MyDrive/Colab Notebooks/mafs_nCop/'
path_nets = '/content/drive/MyDrive/Colab Notebooks/redes/'
path_frequence = '/content/drive/MyDrive/Colab Notebooks/mafs_cBioPortal/MutationFrequence/'

path_mafs_list = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists/'
path_nets_list = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/NET_lists/'

path_mafs_freq_10p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_10p/'
path_mafs_freq_5p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_5p/'
path_mafs_freq_3p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
Gera uma lista com o nome de todos os arquivos do diretório (mafs ou redes)
"""
def file_list_in_dir(path):
    l_files = []
    for _, _, file in os.walk(path):
        l_files = file
    return l_files

"""
Abre todos os mafs do diretório
"""    
def open_mafs(l_mafs_name):
    l_mafs_csv = []
    for maf in l_mafs_name:
        df = pd.read_csv(path+maf, sep='\t', usecols=["Hugo_Symbol", "Tumor_Sample_Barcode"])
        l_mafs_csv.append(df)
    return l_mafs_csv

In [ ]:
# Retorna uma lista de genes do MAF em que aparece mutado mais de uma vez em um mesmo paciente
def listaGenesPacientes(maf):
    ListGenes = []
    for i in open(maf):
        ListPontGene = []
        lista = str(i).split(' ')
        ListPontGene.append(lista[0])
        tamanhoSemRepetidos = len(list(set(lista)))-1
        tamanhoComRepetidos = len(lista)-1
        ListPontGene.append(tamanhoSemRepetidos)
        ListPontGene.append(tamanhoComRepetidos)
        
        if tamanhoSemRepetidos != tamanhoComRepetidos:
            ListGenes.append(ListPontGene[0])

    return ListGenes

def listaGenesPacientesCorreto(maf):
    ListGenes = []
    for i in open(maf):
        ListPontGene = []
        lista = str(i).split(' ')
        ListGenes.append(lista[0])

    return set(ListGenes)

def listaGenesPacientesFreq(l_mafs_frequence, percent, path_mafs_freq):

    for freq in l_mafs_frequence:
        listGenes = []
        dfFrequence = pd.read_csv(path_frequence+freq, sep='\t', usecols=['Gene', 'Freq'])
        dfFrequence['Freq'] = dfFrequence['Freq'].str.rstrip('%')
        dfFrequence['Freq'] = dfFrequence['Freq'].astype('float')
        dfFrequence = dfFrequence[(dfFrequence['Freq']>=percent)]
        listGenes = list(dfFrequence['Gene'])
        escreveListaEmArquivo(listGenes, path_mafs_freq, freq)


# Retorna um dicionario de genes do MAF pontuados e uma lista de todos os genes
def calculaPontGenes(maf):
    DictPontGenes = {}
    ListGenes = []
    for i in open(maf):
        ListPontGene = []
        lista = str(i).split(' ')
        ListPontGene.append(lista[0])
        ListPontGene.append(str(len(lista)-1))
        ListGenes.append(ListPontGene[0])
        DictPontGenes[ListPontGene[0]] = ListPontGene[1]
    
    return set(ListGenes), DictPontGenes

def escreveListaEmArquivo(lista, path, nome):
    nome_arquivo = path + nome
    print(nome_arquivo)
    arquivo = open(nome_arquivo, "a")
    for i in lista:
        arquivo.write(str(i)+'\n')
    arquivo.close()

In [ ]:

"""
Recebe uma lista de de todas os MAFs do diretório e converte todos em uma lista de genes
Saída: um arquivo para cada MAF, com a lista de genes do MAF
"""
def convert_mafnCop_mafGenesList(l_mafs_name):
    l_mafs_csv = []
    for maf in l_mafs_name:
        #ListGenes, DictPontGenes = calculaPontGenes(path_mafs_nCop+maf)
        ListGenes = listaGenesPacientesCorreto(path_mafs_nCop+maf)
        print('\n'+maf)
        print(ListGenes)
        print('tamanho: ' + str(len(ListGenes)))
        escreveListaEmArquivo(ListGenes, path_mafs_list, maf)

"""
Recebe uma lista de de todas as redes do diretório e converte todos em uma lista de genes
Saída: um arquivo para cada rede, com a lista de genes da rede
"""
def converte_net_netGenesList(l_nets_name):
    l_nets_csv = []
    for net in l_nets_name:
        df = pd.read_csv(path_nets+net, sep="\s+", header=None, names=["geneA", "geneB"])
        # Captura todos os genes sem os repetidos de cada coluna
        colunaA = df["geneA"].unique().tolist()
        colunaB = df["geneB"].unique().tolist()
        listaGenes = colunaA + colunaB
        listaGenes = list(set(listaGenes))
        print('\n'+net)
        print(listaGenes)
        print('tamanho: ' + str(len(listaGenes)))
        escreveListaEmArquivo(listaGenes, path_nets_list, net)


In [ ]:
l_mafs_name = file_list_in_dir(path_mafs_nCop)
convert_mafnCop_mafGenesList(l_mafs_name)


In [ ]:
l_nets_name = file_list_in_dir(path_nets)
converte_net_netGenesList(l_nets_name)


In [ ]:
l_mafs_frequence = file_list_in_dir(path_frequence)
listaGenesPacientesFreq(l_mafs_frequence, 3, path_mafs_freq_3p)
listaGenesPacientesFreq(l_mafs_frequence, 5, path_mafs_freq_5p)
listaGenesPacientesFreq(l_mafs_frequence, 10, path_mafs_freq_10p)


/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/ACC_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/BLCA_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/BRCA_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/CESC_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/COAD_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/GBM_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/HNSC_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/KICH_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/KIRC_frequence.txt
/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/MAF_lists_freq_3p/KIRP_freque